In [1]:
import pandas as pd
import numpy as np

In [6]:
# charger le fichier 
data = pd.read_table("./data/SMSSpamCollection", header=None, names= ["type", "message"])

In [7]:
data

,type,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [10]:
x = data["message"]
y = data["type"]

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [9]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
modele_transfo = TfidfVectorizer()
x_train_trans = modele_transfo.fit_transform(x_train)

In [13]:
x_train_trans

<4457x7702 sparse matrix of type '<class 'numpy.float64'>'
	with 59296 stored elements in Compressed Sparse Row format>

In [14]:
modele_svm = LinearSVC()

In [16]:
modele_svm.fit(x_train_trans, y_train)

LinearSVC()

In [18]:
modele_svm.predict(modele_transfo.transform(x_test))

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [19]:
# utilisation d'un pipeline
from sklearn.pipeline import make_pipeline

In [31]:
mon_pipeline = make_pipeline(TfidfVectorizer(ngram_range=(1,2)),LinearSVC())

In [32]:
mon_pipeline.fit(x_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(ngram_range=(1, 3))),
                ('linearsvc', LinearSVC())])

In [33]:
mon_pipeline.predict(x_test)

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [34]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [35]:
accuracy_score(y_test, mon_pipeline.predict(x_test))

0.9919282511210762

In [36]:
from sklearn.model_selection import GridSearchCV

In [37]:
dico_grid = {"tfidfvectorizer__ngram_range": [(1,1), (1,2)],
            "linearsvc__C": [0.1,1,5]}

In [38]:
modele_grid_spam = GridSearchCV(mon_pipeline, dico_grid, scoring="accuracy", cv=4)

In [39]:
modele_grid_spam.fit(x_train, y_train)

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('tfidfvectorizer',
                                        TfidfVectorizer(ngram_range=(1, 3))),
                                       ('linearsvc', LinearSVC())]),
             param_grid={'linearsvc__C': [0.1, 1, 5],
                         'tfidfvectorizer__ngram_range': [(1, 1), (1, 2)]},
             scoring='accuracy')

In [40]:
pd.DataFrame(modele_grid_spam.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_linearsvc__C,param_tfidfvectorizer__ngram_range,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.087682,0.003471,0.024386,0.000494,0.1,"(1, 1)","{'linearsvc__C': 0.1, 'tfidfvectorizer__ngram_...",0.977578,0.973070,0.964093,0.965889,0.970158,0.005444,5
1,0.223066,0.013049,0.042317,0.000784,0.1,"(1, 2)","{'linearsvc__C': 0.1, 'tfidfvectorizer__ngram_...",0.967713,0.965889,0.950628,0.951526,0.958939,0.007895,6
2,0.088594,0.002828,0.023445,0.000333,1,"(1, 1)","{'linearsvc__C': 1, 'tfidfvectorizer__ngram_ra...",0.990135,0.985637,0.983842,0.984740,0.986088,0.002421,2
3,0.231051,0.007624,0.042894,0.001697,1,"(1, 2)","{'linearsvc__C': 1, 'tfidfvectorizer__ngram_ra...",0.990135,0.986535,0.982944,0.983842,0.985864,0.002797,3
4,0.097659,0.002268,0.024624,0.000776,5,"(1, 1)","{'linearsvc__C': 5, 'tfidfvectorizer__ngram_ra...",0.990135,0.985637,0.980251,0.986535,0.985640,0.003537,4
5,0.242164,0.002240,0.043699,0.003058,5,"(1, 2)","{'linearsvc__C': 5, 'tfidfvectorizer__ngram_ra...",0.988341,0.987433,0.985637,0.984740,0.986538,0.001423,1


In [48]:
def filtre_anti_spam(message):
    return mon_pipeline.predict(np.array([message]))

In [54]:
filtre_anti_spam("Free Free Free Free Free Will ü b going to esplanade fr home?")

array(['spam'], dtype=object)